# Lariat Displays and Movies with ParaView

Adam Lyon / Fermilab SCD / September 2015

!! Not complete !! -- **and won't be complete** I had been detailing all of the ParaView steps with python commands passed to `pvpython` in collaboration mode with a backend server and a GUI. It's kinda neat as when you execute a cell, the ParaView GUI reacts accordingly in real time. But in pratice, determining those python commands is very tedious, and in reality, the state files will be much more useful if someone wants to adjust or alter a visualization than the python commands. So I'm re-writing the ParaView parts of this document without the `pvpython` commands. 

The plan is to demonstrate use of ParaView to make some simple, but effective, event displays of Liquid Argon neutrino data with not too much effort. See Jim Kowalkowski's notes at https://github.com/jbkowalkowski/AnalysisWork/wiki/Geant4-visualization-and-LArSoft for more information.

Jim has code (see link above) that can write out event data from Lariat from one event (for now). He generates comma-separated-value files (`.csv`) with this data. I have a script `convertCSV.py` that converts the csv file into VTK PolyData and writes that out to a `.vtp` file that ParaView can load directly. Each row of the csv must contain a 3-D position $(x,y,z)$ and then additional columns of metadata that can be integers, floats, or strings. The script will automatically create vectors out of neighboring columns whose titles end in `x`, `y`, `z`. For example, three columns in a row named `px, py, pz` will be turned into a vector called `p`. Other column metadata are turned into scalars. The script associates the metadata with each point. Each point is also a `VTK_VERTEX` cell, but the cell has no metadata (the ParaView `Point Data To Cell Data` filter can copy the point data to the cells if they are needed). 

## Hit and track information

Let's first look at some basic hit & track information. Jim has a file called `output_po.csv` containing hits and metadata.

In [1]:
!head data20150919/output_po.csv

eid/I,type/S,id/I,index/I,x/F,y/F,z/F,dirx/F,diry/F,dirz/F,p/F
1,pmtrack,0,0,29.4108,-1.02638,50.6487,0.10971,0.035345,-0.993335,0
1,pmtrack,0,1,29.4585,-1.01102,50.217,0.10971,0.035345,-0.993335,0
1,pmtrack,0,2,29.5111,-0.994092,49.7413,0.10971,0.035345,-0.993335,0
1,pmtrack,0,3,29.541,-0.984435,49.4699,0.10971,0.035345,-0.993335,0
1,pmtrack,0,4,29.5606,-0.978127,49.2927,0.10971,0.035345,-0.993335,0
1,pmtrack,0,5,29.5959,-0.966758,48.9731,0.10971,0.035345,-0.993335,0
1,pmtrack,0,6,29.611,-0.961909,48.8369,0.10971,0.035345,-0.993335,0
1,pmtrack,0,7,29.6475,-0.95012,48.5055,0.10971,0.035345,-0.993335,0
1,pmtrack,0,8,29.6612,-0.945724,48.382,0.10971,0.035345,-0.993335,0


The first line is the header with the column name followed by the type (`I` is integer, `F` is float, `S` is string). The types are needed by my `convertCSV.py` script. 

The data contain rows of hits. Each hit row has:
* `eid` The event ID (always 1 since we only have one event)
* `type` The track type (`pmtrack`, `costrack`, `cctrack`) representing the tracking algorithm that produced the track. 
* `id` The ID of the track
* `index` The index of the hit on the track (starts at 0 for new track)
* `x,y,z` The position of the hit
* `dirx, diry, dirz` The track direction at the hit
* `p` Not sure what this is, but it's always zero

We can convert this to the vtp file,

In [2]:
!pvpython convertCSV.py data20150919/output_po.csv

Read File
[('eid', '<i8'), ('type', 'S20'), ('id', '<i8'), ('index', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('dirx', '<f8'), ('diry', '<f8'), ('dirz', '<f8'), ('p', '<f8')]
Assemble polydata
Making array eid
Making array (string) type
Making array id
Making array index
Making array dirx diry dirz
Making array p
Writing output_po.vtp


Let's display this data in ParaView. ParaView can be driven by this Jupyter Notebook by running `pvpython`. I'll use my `ipythonPexpect` Jupyter extension to launch and control pvpython. It sends commands to a command-line application using the python `pexpect` module. 

In [36]:
%load_ext ipythonPexpect

The ipythonPexpect extension is already loaded. To reload it, use:
  %reload_ext ipythonPexpect


We can start a ParaView python client (pvpython) and connect it to the server and the GUI. Using the ParaView collaboration tools, the python client can take control of the GUI. 

In [64]:
%pexpect_spawn_pvpython

Opened connection to pvpython
Python 2.7.10 (default, Jul 14 2015, 19:46:27) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.39)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 

In [65]:
%pexpect_lock

To return to IPython, issue %pexpect_unlock


We are now talking to the `pvpython` process. 

In [66]:
from paraview.simple import *
import connectParaViewJupyter as cpvj

paraview version 4.4.0-26-g1150ae7
ParaView <--> Jupyter Connection
Follow these steps:
   1) Start the ParaView server with "pvserver --multi-clients"
   2) Start the ParaView GUI and connect to the server
   3) In pvpython (not regular python) make the connection and take control of the gui with
        c = connectParaViewJuputer.Connection()


The server and GUI need to be started by hand. Then we can connect this pvpython to the GUI and the server.

In [67]:
c = cpvj.Connection()

Connected to server
Controlling GUI


We need to load the `output_po.vtp` file and let's activate all of the metadata except for the uninteresting `p`.

In [68]:
from paraview.simple import *

In [69]:
povtp = XMLPolyDataReader(FileName='output_po.vtp')
povtp.PointArrayStatus = ['eid', 'type', 'id', 'index', 'dir']

Set up the view and display

In [70]:
renderView1 = GetActiveViewOrCreate('RenderView')
renderView1.ViewSize = [1000, 650]

povtpDisp = Show(povtp, renderView1)
povtpDisp.ColorArrayName = [None, '']
renderView1.ResetCamera()

`XMLPolyDataReader` is an ugly name, let's change it to 'event1`.

In [71]:
RenameSource('event1', povtp)
povtp = FindSource('event1')  # Renaming invalidates the old pointer, re-find it

Let's color the tracks by the reconstruction algorithm (the `type` metadata column): `costrack`, `pmtrack` and `cctrack`. 

Select `type` as the color and then bring up the color map editor window (View menu). Choose `Interpret Values as Categories` and set it up for the different track algorithm types. The following code does the same thing.

In [72]:
ColorBy(povtpDisp, ('POINTS', 'type'))
typeLUT = GetColorTransferFunction('type')
typeLUT.InterpretValuesAsCategories = 1
typeLUT.Annotations = ['cctrack', 'cctrack', 'costrk', 'costrk', 'pmtrack', 'pmtrack'] # Names are here twice, once for value and again for label
typeLUT.IndexedColors = [0.90, 0.10, 0.11, 0.215, 0.50, 0.72, 0.30, 0.686, 0.3]

typeLUTColorBar = GetScalarBar( typeLUT, renderView1 )
typeLUTColorBar.Position = [0.9, 0.6]  # Lower left corner is 0,0
typeLUTColorBar.Position2 = [0.2, 0.3]  # 2nd number seems to be height, 1st is ignored
typeLUTColorBar.Title = 'Track Algo'
typeLUTColorBar.ComponentTitle = ''
povtpDisp.SetScalarBarVisibility(renderView1, True)

True


Let's move the camera to a good spot

In [73]:
renderView1.CameraPosition = [76, 55, -26]
renderView1.CameraFocalPoint = [-13, -49, 83]
renderView1.CameraViewUp = [0.805, -0.0704, 0.5884]
renderView1.CameraParallelScale = 38.0
RenderAllViews()

Let's make a screen shot.

In [74]:
SaveScreenshot('outputpo_1.png', view=renderView1, magnification=1, quality=100)

True


![Image](outputpo_1.png)

Zoom in a bit

In [75]:
renderView1.CameraPosition =  [28.573629998647103, 27.04210690132971, 49.76810737297924]
renderView1.CameraFocalPoint =  [28.573629998647103, -232.69558720908339, 49.76810737297924]
renderView1.CameraViewUp =  [0.0, 0.0, 1.0]
RenderAllViews()

In [76]:
SaveScreenshot('outputpo_2.png', view=renderView1, magnification=1, quality=100)

True


![Image](outputpo_2.png)

What you see is that the pmtracks and the costrks follow each other, except for one costrk (lower right in picture above) that does not have a pmtrack counterpart. The cctracks are very short and rarely appear. 

Let's save a state file so we can get back here if necessary. 

In [77]:
SaveState("outputpo_2.pvsm")

Let's now put this in context by showing a picture of the detector and the tracks within.

--- More to do here ---

In [62]:
%pexpect_unlock

UsageError: No connection

In [63]:
%pexpect_close

UsageError: No connection

## Track/hit movie

Make a movie showing the time evolution of a Lariat track. Jim Kowalkowski created some CSV files from Lariat data. 

The file `output_po.csv` has information on hits on tracks, including the $x,y,z$ position.

In [32]:
!head data20150919/output_po.csv

eid/I,type/S,id/I,index/I,x/F,y/F,z/F,dirx/F,diry/F,dirz/F,p/F
1,pmtrack,0,0,29.4108,-1.02638,50.6487,0.10971,0.035345,-0.993335,0
1,pmtrack,0,1,29.4585,-1.01102,50.217,0.10971,0.035345,-0.993335,0
1,pmtrack,0,2,29.5111,-0.994092,49.7413,0.10971,0.035345,-0.993335,0
1,pmtrack,0,3,29.541,-0.984435,49.4699,0.10971,0.035345,-0.993335,0
1,pmtrack,0,4,29.5606,-0.978127,49.2927,0.10971,0.035345,-0.993335,0
1,pmtrack,0,5,29.5959,-0.966758,48.9731,0.10971,0.035345,-0.993335,0
1,pmtrack,0,6,29.611,-0.961909,48.8369,0.10971,0.035345,-0.993335,0
1,pmtrack,0,7,29.6475,-0.95012,48.5055,0.10971,0.035345,-0.993335,0
1,pmtrack,0,8,29.6612,-0.945724,48.382,0.10971,0.035345,-0.993335,0


The file `output_ht.csv` has raw ADC and TDC information.

In [33]:
!head data20150919/output_ht.csv

eid/I,type/S,id/I,peaktime/F,chan/F,peakamp/F,sumadc/F,view/I,wire/S
1,pmtrack,0,1506.45,132,48.487,724.583,0,C:0 T:0 P:0 W:132
1,pmtrack,0,1501.37,131,19.7431,461.965,0,C:0 T:0 P:0 W:131
1,pmtrack,0,1512.35,130,8.46552,194.732,0,C:0 T:0 P:0 W:130
1,pmtrack,0,1508.53,369,13.2191,192.482,1,C:0 T:0 P:1 W:129
1,pmtrack,0,1513.47,129,8.85346,135.981,0,C:0 T:0 P:0 W:129
1,pmtrack,0,1519.69,368,11.4093,173.319,1,C:0 T:0 P:1 W:128
1,pmtrack,0,1517.19,128,8.6917,125.778,0,C:0 T:0 P:0 W:128
1,pmtrack,0,1521,367,10.6517,178.272,1,C:0 T:0 P:1 W:127
1,pmtrack,0,1520.57,127,7.12778,103.499,0,C:0 T:0 P:0 W:127


We need to merge the files so that we have $x,y,z$ and the hit information in one place.

Look at the lengths of the files

In [34]:
!wc data20150919/output_po.csv

     796     796   57087 data20150919/output_po.csv


In [35]:
!wc data20150919/output_ht.csv

     321    1281   19032 data20150919/output_ht.csv


There are many kinds of tracks in `output_po.csv`, but only one in `output_ht.csv` (pmtrack). Lets focus on those.

In [36]:
!grep pmtrack data20150919/output_po.csv | wc

     320     320   23127


This has one less than `output_ht.csv` due to the header in the latter. So it looks like the hits should line up.

Let's load them in with pandas and see if we can merge them.

In [37]:
import pandas as pd

In [38]:
pd.__version__

u'0.17.0rc1'

In [39]:
po = pd.read_csv("data20150919/output_po.csv")

In [40]:
po.head()

,eid/I,type/S,id/I,index/I,x/F,y/F,z/F,dirx/F,diry/F,dirz/F,p/F
0,1,pmtrack,0,0,29.4108,-1.026380,50.6487,0.10971,0.035345,-0.993335,0
1,1,pmtrack,0,1,29.4585,-1.011020,50.2170,0.10971,0.035345,-0.993335,0
2,1,pmtrack,0,2,29.5111,-0.994092,49.7413,0.10971,0.035345,-0.993335,0
3,1,pmtrack,0,3,29.5410,-0.984435,49.4699,0.10971,0.035345,-0.993335,0
4,1,pmtrack,0,4,29.5606,-0.978127,49.2927,0.10971,0.035345,-0.993335,0


In [41]:
ht = pd.read_csv("data20150919/output_ht.csv")

In [42]:
ht.head()

,eid/I,type/S,id/I,peaktime/F,chan/F,peakamp/F,sumadc/F,view/I,wire/S
0,1,pmtrack,0,1506.45,132,48.48700,724.583,0,C:0 T:0 P:0 W:132
1,1,pmtrack,0,1501.37,131,19.74310,461.965,0,C:0 T:0 P:0 W:131
2,1,pmtrack,0,1512.35,130,8.46552,194.732,0,C:0 T:0 P:0 W:130
3,1,pmtrack,0,1508.53,369,13.21910,192.482,1,C:0 T:0 P:1 W:129
4,1,pmtrack,0,1513.47,129,8.85346,135.981,0,C:0 T:0 P:0 W:129


In [43]:
len(ht), len(po)

(320, 795)

What are the kinds of tracks?

In [44]:
pd.unique( ht['type/S'] )

array(['pmtrack'], dtype=object)

In [45]:
pd.unique( po['type/S'])

array(['pmtrack', 'cctrack', 'costrk'], dtype=object)

So we only want the `pmtrack`.

In [46]:
popm = po[ po['type/S'] == 'pmtrack' ]

In [47]:
len(popm)

320

That matches `ht`

In [48]:
popm.head()

,eid/I,type/S,id/I,index/I,x/F,y/F,z/F,dirx/F,diry/F,dirz/F,p/F
0,1,pmtrack,0,0,29.4108,-1.026380,50.6487,0.10971,0.035345,-0.993335,0
1,1,pmtrack,0,1,29.4585,-1.011020,50.2170,0.10971,0.035345,-0.993335,0
2,1,pmtrack,0,2,29.5111,-0.994092,49.7413,0.10971,0.035345,-0.993335,0
3,1,pmtrack,0,3,29.5410,-0.984435,49.4699,0.10971,0.035345,-0.993335,0
4,1,pmtrack,0,4,29.5606,-0.978127,49.2927,0.10971,0.035345,-0.993335,0


Well, one final check can be how many hits there are per track (we'll count rows -- just need a column to do that with, so choose the event number - doesn't matter which column we choose)

In [49]:
ht[ ['id/I', 'eid/I'] ].groupby('id/I').count()

,eid/I
id/I,
0,203
1,34
2,59
65539,24


In [50]:
popm[ ['id/I', 'eid/I'] ].groupby('id/I').count()

,eid/I
id/I,
0,203
1,34
2,59
65539,24


They look the same! Let's merge them for conversion to VTK

We need to add an index to the ht hits. How to do this?

In [51]:
ht['index/I'] = ht.groupby(['id/I']).cumcount()

In [52]:
ht.head()

,eid/I,type/S,id/I,peaktime/F,chan/F,peakamp/F,sumadc/F,view/I,wire/S,index/I
0,1,pmtrack,0,1506.45,132,48.48700,724.583,0,C:0 T:0 P:0 W:132,0
1,1,pmtrack,0,1501.37,131,19.74310,461.965,0,C:0 T:0 P:0 W:131,1
2,1,pmtrack,0,1512.35,130,8.46552,194.732,0,C:0 T:0 P:0 W:130,2
3,1,pmtrack,0,1508.53,369,13.21910,192.482,1,C:0 T:0 P:1 W:129,3
4,1,pmtrack,0,1513.47,129,8.85346,135.981,0,C:0 T:0 P:0 W:129,4


The index columns should match in the two datasets

In [53]:
all(ht['index/I'] == popm['index/I'])

True

Let's merge!

In [54]:
htpm = pd.merge(ht, popm, on=['id/I', 'index/I'])

In [55]:
htpm.drop(['eid/I_x', 'eid/I_y', 'type/S_x', 'type/S_y'], axis=1, inplace=True)

In [56]:
htpm.head()

,id/I,peaktime/F,chan/F,peakamp/F,sumadc/F,view/I,wire/S,index/I,x/F,y/F,z/F,dirx/F,diry/F,dirz/F,p/F
0,0,1506.45,132,48.48700,724.583,0,C:0 T:0 P:0 W:132,0,29.4108,-1.026380,50.6487,0.10971,0.035345,-0.993335,0
1,0,1501.37,131,19.74310,461.965,0,C:0 T:0 P:0 W:131,1,29.4585,-1.011020,50.2170,0.10971,0.035345,-0.993335,0
2,0,1512.35,130,8.46552,194.732,0,C:0 T:0 P:0 W:130,2,29.5111,-0.994092,49.7413,0.10971,0.035345,-0.993335,0
3,0,1508.53,369,13.21910,192.482,1,C:0 T:0 P:1 W:129,3,29.5410,-0.984435,49.4699,0.10971,0.035345,-0.993335,0
4,0,1513.47,129,8.85346,135.981,0,C:0 T:0 P:0 W:129,4,29.5606,-0.978127,49.2927,0.10971,0.035345,-0.993335,0


In [57]:
len(htpm)

320

Write it out!

In [58]:
htpm.to_csv('output_htpm.csv', index=False)   # index=False says don't write the row name (it's empty)

In [59]:
!head output_htpm.csv

id/I,peaktime/F,chan/F,peakamp/F,sumadc/F,view/I,wire/S,index/I,x/F,y/F,z/F,dirx/F,diry/F,dirz/F,p/F
0,1506.45,132,48.487,724.583,0,C:0 T:0 P:0 W:132,0,29.4108,-1.02638,50.6487,0.10971,0.035345,-0.993335,0
0,1501.37,131,19.7431,461.965,0,C:0 T:0 P:0 W:131,1,29.4585,-1.01102,50.217,0.10971,0.035345,-0.993335,0
0,1512.35,130,8.46552,194.732,0,C:0 T:0 P:0 W:130,2,29.5111,-0.994092,49.7413,0.10971,0.035345,-0.993335,0
0,1508.53,369,13.2191,192.482,1,C:0 T:0 P:1 W:129,3,29.541,-0.984435,49.4699,0.10971,0.035345,-0.993335,0
0,1513.47,129,8.85346,135.981,0,C:0 T:0 P:0 W:129,4,29.5606,-0.978127,49.2927,0.10971,0.035345,-0.993335,0
0,1519.69,368,11.4093,173.319,1,C:0 T:0 P:1 W:128,5,29.5959,-0.966758,48.9731,0.10971,0.035345,-0.993335,0
0,1517.19,128,8.6917,125.778,0,C:0 T:0 P:0 W:128,6,29.611,-0.961909,48.8369,0.10971,0.035345,-0.993335,0
0,1521.0,367,10.6517,178.272,1,C:0 T:0 P:1 W:127,7,29.6475,-0.95012,48.5055,0.10971,0.035345,-0.993335,0
0,1520.57,127,7.12778,103.499,0,C:0 T:0 P:0 W:127,8,

Now convert to VTK

In [60]:
!pvpython convertCSV.py output_htpm.csv

Read File
[('id', '<i8'), ('peaktime', '<f8'), ('chan', '<f8'), ('peakamp', '<f8'), ('sumadc', '<f8'), ('view', '<i8'), ('wire', 'S20'), ('index', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('dirx', '<f8'), ('diry', '<f8'), ('dirz', '<f8'), ('p', '<f8')]
Assemble polydata
Making array id
Making array peaktime
Making array chan
Making array peakamp
Making array sumadc
Making array view
Making array (string) wire
Making array index
Making array dirx diry dirz
Making array p
Writing output_htpm.vtp


--- More to do here ---